# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [1]:
# Imports
import pandas as pd
import numpy as np

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [2]:
recipes = pd.read_csv("recipes.tsv", sep="\t")
recipes

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...


In [3]:
ratings = pd.read_csv("recipes_rated.csv")
ratings = ratings[["recipe_slug", "ratings_raw"]]
ratings

,recipe_slug,ratings_raw
0,falafel,5
1,spamburger,3
2,bacon-fried-rice,3
3,chicken-fingers,1
4,apple-crisp,4
...,...,...
95,bacon-mac-and-cheese,3
96,chicken-alfredo-lasagna,5
97,classic-beef-lasagna,3
98,vegetarian-mushroom-lasagna,2


In [4]:
ratings["rating"] = (ratings["ratings_raw"] - 1) * 0.25

In [5]:
ratings = ratings.drop("ratings_raw", axis=1)
ratings

,recipe_slug,rating
0,falafel,1.00
1,spamburger,0.50
2,bacon-fried-rice,0.50
3,chicken-fingers,0.00
4,apple-crisp,0.75
...,...,...
95,bacon-mac-and-cheese,0.50
96,chicken-alfredo-lasagna,1.00
97,classic-beef-lasagna,0.50
98,vegetarian-mushroom-lasagna,0.25


In [6]:
ratings.to_csv("ratings.tsv", sep="\t", index=False)

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [7]:
# YOUR CHANGES HERE

recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")
recipe_tags


,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack
...,...,...
747,bacon-souffle,breakfast
748,bacon-souffle,brunch
749,bacon-souffle,cheese
750,bacon-souffle,eggs


In [8]:
encoded_recipe_tags = recipe_tags.pivot_table(
    index="recipe_slug", 
    columns="recipe_tag", 
    aggfunc=lambda x: 1,
    fill_value=0
)
encoded_recipe_tags

recipe_tag,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,bacon,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
recipe_slug,,,,,,,,,,,,,,,,,,,,,
almond-chip-cookies,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
almond-croissants,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
apple-crisp,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
apple-crumble,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
apple-pie,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tempura-udon,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
udon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
vanilla-souffle,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
features = pd.merge(ratings[["recipe_slug"]], encoded_recipe_tags, how="left", on=["recipe_slug"])
features

,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
features.insert(1, "bias", 1)
features

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
features.iloc[:, 2:] = features.iloc[:, 2:].astype(int)
features

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [12]:
features.to_csv("features.tsv", sep="\t", index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [86]:
# YOUR CHANGES HERE
ridge_data = ratings.merge(features, on="recipe_slug", how="left")
ridge_X = ridge_data.iloc[:, 1:].drop("rating", axis=1)
ridge_y = ridge_data[["rating"]]

In [87]:
ridge_X

,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [88]:
ridge_y

,rating
0,1.00
1,0.50
2,0.50
3,0.00
4,0.75
...,...
95,0.50
96,1.00
97,0.50
98,0.25


In [89]:
from sklearn.linear_model import Ridge

alpha=1.0
ridge_model = Ridge(alpha=alpha, fit_intercept=False)

In [90]:
ridge_model.fit(ridge_X, ridge_y)

,alpha,1.0
,fit_intercept,False
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [91]:
ridge_X.shape

(100, 297)

In [92]:
ridge_y.shape

(100, 1)

In [93]:
ridge_coefficients = np.ravel(ridge_model.coef_)
ridge_coefficients

array([ 4.77864768e-01,  1.07172340e-01,  4.41227327e-02,  1.87961806e-02,
        6.91883104e-02,  6.84119195e-02, -3.09787243e-02, -4.93885004e-02,
       -1.27728812e-01, -1.93929002e-02,  7.23673740e-02, -5.94520038e-02,
        1.17549810e-01, -4.78938452e-02,  3.00875352e-02,  4.74126562e-02,
        2.10032229e-02,  4.50289435e-02, -3.63123877e-02,  4.50289435e-02,
       -1.90778632e-04, -5.53479421e-03,  5.55162346e-03,  5.55162346e-03,
        5.21125925e-02, -4.26430488e-02, -5.25411624e-02,  5.84657638e-02,
        5.64062155e-02, -8.25198212e-02,  3.50918246e-02, -1.46162282e-01,
        6.40629315e-02,  1.04594947e-01,  1.95055153e-02,  2.98358326e-02,
       -4.89803568e-02,  2.98256714e-02,  5.55162346e-03, -3.71835966e-02,
        1.33313536e-02, -1.18475664e-01, -4.80473718e-02, -2.41404346e-03,
        6.54701620e-02,  1.95055153e-02,  3.78821244e-02,  4.50289435e-02,
       -1.08221029e-01,  3.80729030e-02,  5.88265873e-02,  5.82821355e-02,
        6.07993933e-03,  

In [94]:
ridge_coefficients.shape

(297,)

In [96]:
ridge_X_cols = ridge_X.columns
len(ridge_X_cols)

297

In [97]:
model = pd.DataFrame({
    "recipe_tag": ridge_X_cols, 
    "coefficient": np.ravel(ridge_coefficients)
})
model.head()

,recipe_tag,coefficient
0,bias,0.477865
1,alfredo,0.107172
2,almond,0.044123
3,american,0.018796
4,appetizer,0.069188


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

In [98]:
# YOUR CHANGES HERE

model.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [99]:
# YOUR CHANGES HERE

ridge_predictions.shape

(100,)

In [100]:
estimates = pd.DataFrame({
    "recipe_slug": features["recipe_slug"], 
    "score_estimate": ridge_predictions
})
estimates.head()

,recipe_slug,score_estimate
0,falafel,0.904723
1,spamburger,0.495621
2,bacon-fried-rice,0.512735
3,chicken-fingers,0.107582
4,apple-crisp,0.760933


In [101]:
estimates.to_csv("estimates.tsv", sep="\t", index=False)

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [102]:
# YOUR CHANGES HERE

ratings.shape

(100, 2)

In [103]:
linucb_df = ratings.merge(features, on="recipe_slug", how="inner")
linucb_df

,recipe_slug,rating,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1.00,1,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0.50,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0.50,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0.00,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0.75,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0.50,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1.00,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0.50,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0.25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [105]:
ridge_alpha = alpha
ridge_alpha

1.0

In [108]:
X_once = linucb_df.iloc[:, 2:]
X_once.head()

,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [109]:
A_inv = np.linalg.inv(X_once.T @ X_once + ridge_alpha * np.eye(X_once.shape[1]))
A_inv

array([[ 2.22918954e-01, -1.66377277e-02, -2.22940923e-02, ...,
         9.22292229e-03, -5.68782445e-03,  3.19190818e-03],
       [-1.66377277e-02,  7.81520510e-01,  1.29378544e-03, ...,
         2.30782196e-03,  3.46873003e-03,  8.68663569e-04],
       [-2.22940923e-02,  1.29378544e-03,  5.24951420e-01, ...,
         1.46688972e-02, -7.86236016e-03,  5.96794999e-03],
       ...,
       [ 9.22292229e-03,  2.30782196e-03,  1.46688972e-02, ...,
         8.73918550e-01,  5.61992064e-02, -1.97989378e-03],
       [-5.68782445e-03,  3.46873003e-03, -7.86236016e-03, ...,
         5.61992064e-02,  7.75554853e-01,  1.76591765e-03],
       [ 3.19190818e-03,  8.68663569e-04,  5.96794999e-03, ...,
        -1.97989378e-03,  1.76591765e-03,  8.55288720e-01]],
      shape=(297, 297))

In [110]:
all_recipes_X = features[X_once.columns].to_numpy(dtype=float)
uncertainty = np.sqrt(np.einsum("ij,jk,ik->i", all_recipes_X, A_inv, all_recipes_X))

In [113]:
# Merge with the existing estimates and create the upper bounds
linucb_alpha = 2.0
upper_bounds = features[["recipe_slug"]].merge(estimates, on="recipe_slug", how="left")
upper_bounds["score_bound"] = upper_bounds["score_estimate"] + linucb_alpha * uncertainty

In [114]:
upper_bounds.head()

,recipe_slug,score_estimate,score_bound
0,falafel,0.904723,2.670978
1,spamburger,0.495621,2.389957
2,bacon-fried-rice,0.512735,2.378892
3,chicken-fingers,0.107582,1.901698
4,apple-crisp,0.760933,2.537275


In [117]:
bounds = upper_bounds[["recipe_slug", "score_bound"]]
bounds.head()

,recipe_slug,score_bound
0,falafel,2.670978
1,spamburger,2.389957
2,bacon-fried-rice,2.378892
3,chicken-fingers,1.901698
4,apple-crisp,2.537275


In [118]:
bounds.to_csv("bounds.tsv", sep="\t", index=False)

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [133]:
# YOUR CHANGES HERE

recipe_columns = linucb_df.columns
recipe_columns

Index(['recipe_slug', 'rating', 'bias', 'alfredo', 'almond', 'american',
       'appetizer', 'appetizers', 'apple', 'asiancuisine',
       ...
       'udonnoodles', 'vanilla', 'vanillaicecream', 'vegan', 'vegetables',
       'vegetarian', 'warm', 'whippedcream', 'winter', 'yeastdough'],
      dtype='object', length=299)

In [129]:
# Candidate pool matrices or vectors
slugs = linucb_df["recipe_slug"].to_numpy()
rewards_vector = linucb_df["rating"].to_numpy(dtype=float)

In [146]:
# LinUCB setup
n, p = X_once.shape
T = min(100, n)

# LinUCB (single shared model) state: A, b
A = ridge_alpha * np.eye(p)
b = np.zeros(p, dtype=float)

# Avoid re-recommending the same recipe
loop_available = np.ones(n, dtype=bool)

# List of dictionaries:
records = []

# Loop df
loop_df = features[X_once.columns]
loop_df.head()


,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [148]:
for t in range(T):
    # Current posterior
    loop_A_inv = np.linalg.inv(A)
    loop_theta = loop_A_inv @ b
    X_all = features[X_once.columns]

    # Compute LinUCB upper bound for all available candidates:
    # estimate = X @ theta
    loop_est = X_all @ loop_theta
    loop_uncertainty = np.sqrt(np.einsum("ij,jk,ik->i", X_all, loop_A_inv, X_all))
    loop_ucb = loop_est + linucb_alpha * loop_uncertainty

    # Mask out chosen items
    loop_ucb_masked = np.where(loop_available, loop_ucb, -np.inf)

    # Pick the best index
    loop_max_index = int(np.argmax(loop_ucb_masked))
    if not np.isfinite(loop_ucb_masked[loop_max_index]):
        break

    # Observe the reward from the ratings vector
    loop_reward = float(rewards_vector[loop_max_index])

    # Record in order
    records.append({
        "recipe_slug": slugs[loop_max_index], 
        "score_bound": float(loop_ucb[loop_max_index]), 
        "reward": loop_reward
    })

In [150]:
recommendations = pd.DataFrame(records)
recommendations.head()

,recipe_slug,score_bound,reward
0,apple-crumble,7.483315,0.75
1,apple-crumble,7.483315,0.75
2,apple-crumble,7.483315,0.75
3,apple-crumble,7.483315,0.75
4,apple-crumble,7.483315,0.75


In [152]:
recommendations.to_csv("recommendations.tsv", sep="\t", index=False)

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.